    0000000_16_230203_001_final_cluster_happy_parking_map_before
     
    data :  kyungki_region.json(경계), 230128_cluster_map_lat.csv(경계 좌표)
            k_happy_df.csv(행복주택),korea_parking.csv(주차장)
            
    save : kyungki_pop.html
           230203_result_before.html
           
    목차 
        0) package load
        1) 경기도 시각화 
           - 경계 
           - 경기도 시구 : 42 개 좌표 -> 분석 대상 : 경기도 16개 좌표

        2) 행복주택 data/좌표
        3) 전국주차장 data/좌표
           - 전국 주차장 -> 경기도 주차장 -> 전처리(reset index)
        4) 시각화 
           - base map -> 경기도 경계 -> 16개 지구 color map -> 주차장 -> 행복지구 -> 16개지구 중심 원표시 -> total map
        5) save

In [1]:
#0) package load
import requests
import json
import os
import math
import time
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
import matplotlib.pyplot as plt

from shapely.geometry import mapping, shape, Point, Polygon, LineString
from folium import plugins
from folium.plugins import MarkerCluster, HeatMap
print('슝~')

슝~


In [2]:
# 1) 경기도 경계 시각화 
k_geo_data_path = '../002_data_dir/001_map_data/kyungki_region.json'
k_geo_data = json.load(
    open(k_geo_data_path, encoding="utf-8")
)

print('슝~')

슝~


In [3]:
# 1) 경기도 시구 : 42 개 좌표 -> 분석 대상 : 경기도 16개 좌표
k_map_df = pd.read_csv("../002_data_dir/003_search_data/230128_cluster_map_lat.csv")
k_16_df = k_map_df[k_map_df['socar_data'] == 1]
k_16_df.loc[:,'adm_cd'] = k_16_df.loc[:,'adm_cd'].astype(str)
k_16_df.head()

/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


,adm_cd,adm_nm,socar_data,cluster,char,lon,lat,adm_nm1,adm_nm2
1,41281,고양시 덕양구,1,1.0,A,126.878632,37.655810,고양시,덕양구
2,41285,고양시 일산동구,1,1.0,A,126.797558,37.679865,고양시,일산동구
3,41287,고양시 일산서구,1,1.0,A,126.727983,37.680205,고양시,일산서구
5,41210,광명시,1,2.0,B,126.866708,37.475750,광명시,NaN
7,41310,구리시,1,2.0,B,127.131864,37.591625,구리시,NaN


In [4]:
# 2) 행복주택 data
path = '../002_data_dir/003_search_data/k_happy_df.csv'
happy_house_df = pd.read_csv(path)
happy_house_df.head()  

,지역,단지,세대수,주소,위도,경도,name1,name2
0,고양시 덕양구,삼송11단지,832,경기도 고양시 덕양구 원흥1로 25,37.650992,126.865491,고양시,덕양구
1,고양시 덕양구,고양지축 A-1블록 신혼희망타운 행복주택,250,경기도 고양시 덕양구 오부자로 99,37.653331,126.921037,고양시,덕양구
2,고양시 덕양구,고양지축 A-4BL 행복주택,473,경기도 고양시 덕양구 오부자로 43,37.654038,126.914946,고양시,덕양구
3,고양시 덕양구,고양삼송 LH원흥마을13단지,947,경기도 고양시 덕양구 원흥1로 35,37.652631,126.868888,고양시,덕양구
4,고양시 덕양구,서정마을LH12단지,276,경기도 고양시 덕양구 서정마을2로 17,37.621215,126.847648,고양시,덕양구


In [5]:
# 3) 전국주차장 Data / 좌표
paking_station_info_df = pd.read_csv("../002_data_dir/003_search_data/korea_parking.csv")
paking_station_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14178 entries, 0 to 14177
Data columns (total 50 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID                        14178 non-null  object 
 1   LCLAS_NM                  14178 non-null  object 
 2   MLSFC_NM                  14178 non-null  object 
 3   FCLTY_NM                  14178 non-null  object 
 4   CTPRVN_NM                 11187 non-null  object 
 5   SIGNGU_NM                 11124 non-null  object 
 6   LEGALDONG_CD              11050 non-null  object 
 7   LEGALDONG_NM              11087 non-null  object 
 8   ADSTRD_CD                 11041 non-null  object 
 9   ADSTRD_NM                 11081 non-null  object 
 10  RDNMADR_CD                11002 non-null  float64
 11  RDNMADR_NM                11218 non-null  object 
 12  ZIP_NO                    10947 non-null  float64
 13  GID_CD                    12645 non-null  object 
 14  FCLTY_

In [6]:
#3) 경기도 주차장 Data / 좌표
k_parking = paking_station_info_df[paking_station_info_df['PROVD_INSTT_NM1'] == '경기도']
len(k_parking)

2164

In [7]:
#3) 경기도 주차장 Data 전처리 .. null 제거
k_parking = k_parking[k_parking['FCLTY_LO'].notnull()]
k_parking = k_parking.reset_index(drop=True)
len(k_parking)

1937

In [8]:
#4) 시각화 : base map 지정
# 중심부 위도, 경도
center = [37.541, 126.986]

# map이 서울 중심부에 위치하고, zoom 레벨은 10으로 시작 합니다.
map = folium.Map(
    location=center,
    zoom_start=10,
    )

print('슝~')

슝~


In [9]:
#4) 시각화 : 경기도 경계
folium.GeoJson(
    k_geo_data,
    name='경기도'
).add_to(map)

In [10]:
#4) 시각화 : 분석대상 16개 지구
folium.Choropleth(
    geo_data=k_geo_data,
    name="cluster map",
    data=k_16_df,
    columns=["adm_cd", "cluster"],
    key_on="feature.properties.ADM_SECT_C",
    fill_color='Accent',
    fill_opacity=0.7,
    line_opacity=0.2,
    line_color='white', 
    nan_fill_color='blue',
    nan_fill_opacity =0,
    line_weight=0,
    highlight=False, 
    smooth_factor=1.0,
    legend_name="cluster"
).add_to(map)


In [11]:
#4) 시각화 : 주차장
marker_cluster = MarkerCluster().add_to(map)

# 주차장 이름, 위도, 경도를 가져와 maker 객체로 추가
for index_num in range(len(k_parking)):
    try:
        folium.Marker(
            location=[k_parking.loc[index_num, "FCLTY_LA"], k_parking.loc[index_num, "FCLTY_LO"]],
            popup=k_parking.loc[index_num, "FCLTY_NM"],
            icon=folium.Icon(color="cadetblue", icon="ok")
        ).add_to(marker_cluster)
    except:
        pass

print('슝~')

슝~


In [12]:
#4) 시각화 : 경기도 대상 16개지구 중심에 대한 원표시
marker_cluster = MarkerCluster().add_to(map)

#maker 객체 추가
for idx, row in k_16_df.iterrows():
    try:

        folium.CircleMarker(
            location=[k_16_df.loc[idx, "lat"], k_16_df.loc[idx, "lon"]],
            color='tomato',
            radius=50,
            tooltip = 'zone'           
        ).add_to(marker_cluster)
    except:
        pass

print('슝~')

슝~


In [13]:
#4) 시각화 :  toal map 표시
# map

In [14]:
# #5) map save 
# map.save("../003_map_xml/230203_result_before.html")